# SETTINGS

This notebook performs initial data processing:
- importing the raw data
- converting feature types
- merging some data.frames
- saving data as two CSV files: `orders.csv` and `items.csv`.

A detailed walkthrough of the code covering the key steps is provided in [this blog post](https://kozodoi.me/python/time%20series/demand%20forecasting/competitions/2020/07/27/demand-forecasting.html).

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
##### LIBRARIES

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats

import os
import time
import datetime
import random
import multiprocessing
import pickle
import warnings
import gc
import sys

In [6]:
##### MODULES
sys.path.append('/content/drive/MyDrive/DMC_2020/source_code/codes')  

from data_prep import print_factor_levels, print_missings, find_constant_features, split_nested_features
from versioning import save_csv_version

In [7]:
##### SETTINGS

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use('dark_background')
%matplotlib inline
gc.enable()

# DATA IMPORT

In [8]:
##### IMPORT

infos  = pd.read_csv('/content/drive/MyDrive/DMC_2020/source_code/data/raw/infos.csv',  sep = '|')
items  = pd.read_csv('/content/drive/MyDrive/DMC_2020/source_code/data/raw/items.csv',  sep = '|')
orders = pd.read_csv('/content/drive/MyDrive/DMC_2020/source_code/data/raw/orders.csv', sep = '|')

print(infos.shape)
print(items.shape)
print(orders.shape)

(10463, 3)
(10463, 8)
(2181955, 5)


In [9]:
infos.head()

,itemID,simulationPrice,promotion
0,1,3.43,NaN
1,2,9.15,NaN
2,3,14.04,NaN
3,4,14.10,NaN
4,5,7.48,NaN


In [10]:
items.head()

,itemID,brand,manufacturer,customerRating,category1,category2,category3,recommendedRetailPrice
0,1,0,1,4.38,1,1,1,8.84
1,2,0,2,3.00,1,2,1,16.92
2,3,0,3,5.00,1,3,1,15.89
3,4,0,2,4.44,1,2,1,40.17
4,5,0,2,2.33,1,1,1,17.04


In [11]:
orders.head()

,time,transactID,itemID,order,salesPrice
0,2018-01-01 00:01:56,2278968,450,1,17.42
1,2018-01-01 00:01:56,2278968,83,1,5.19
2,2018-01-01 00:07:11,2255797,7851,2,20.47
3,2018-01-01 00:09:24,2278968,450,1,17.42
4,2018-01-01 00:09:24,2278968,83,1,5.19


# PROCESSING

### MERGE INFOS AND ITEMS

In [12]:
##### MERGER

print(infos.shape)
print(items.shape)
items = pd.merge(infos, items, on = 'itemID', how = 'left')
print(items.shape)
del infos

(10463, 3)
(10463, 8)
(10463, 10)


### CONVERT FEATURE TYPES

In [13]:
print('-' * 50)
print(items.dtypes)
print('-' * 50)
print(orders.dtypes)
print('-' * 50)

--------------------------------------------------
itemID                      int64
simulationPrice           float64
promotion                  object
brand                       int64
manufacturer                int64
customerRating            float64
category1                   int64
category2                   int64
category3                   int64
recommendedRetailPrice    float64
dtype: object
--------------------------------------------------
time           object
transactID      int64
itemID          int64
order           int64
salesPrice    float64
dtype: object
--------------------------------------------------


In [14]:
# items
for var in ['itemID', 'brand', 'manufacturer', 'category1', 'category2', 'category3']:
    items[var] = items[var].astype('str').astype('object') 
    
# orders
for var in ['transactID', 'itemID']:
    orders[var] = orders[var].astype('str').astype('object') 
    
# dates
orders['time'] = pd.to_datetime(orders['time'].astype('str'), infer_datetime_format = True)

### CHECK FEATURES

In [15]:
print_factor_levels(items, top = 3)

Found 7 categorical features.

------------------------------
itemID: 10463 unique values
------------------------------
1       0.000096
6990    0.000096
6972    0.000096
Name: itemID, dtype: float64
------------------------------

------------------------------
promotion: 85 unique values
------------------------------
NaN           0.823855
2018-07-04    0.016152
2018-07-06    0.013763
Name: promotion, dtype: float64
------------------------------

------------------------------
brand: 275 unique values
------------------------------
0      0.500048
101    0.023129
82     0.018924
Name: brand, dtype: float64
------------------------------

------------------------------
manufacturer: 253 unique values
------------------------------
86     0.045972
159    0.042818
164    0.033260
Name: manufacturer, dtype: float64
------------------------------

------------------------------
category1: 8 unique values
------------------------------
2    0.224410
1    0.174233
8    0.165727
Name: cat

In [16]:
print_factor_levels(orders, top = 3)

Found 2 categorical features.

------------------------------
transactID: 2076066 unique values
------------------------------
2280569    0.000088
2258169    0.000085
2280300    0.000080
Name: transactID, dtype: float64
------------------------------

------------------------------
itemID: 9840 unique values
------------------------------
5035    0.004112
5117    0.003695
7789    0.003398
Name: itemID, dtype: float64
------------------------------



In [18]:
find_constant_features(items)

No constant features found.


In [19]:
find_constant_features(orders)

No constant features found.


### MISSING VALUES

In [20]:
# change zeros to NA where relvant
items.loc[items['brand']          == '0', 'brand']        = np.nan
items.loc[items['customerRating'] == 0, 'customerRating'] = np.nan

In [21]:
print_missings(items)

Found 3 features with missing values.


,Total,Percent
promotion,8620,0.823855
customerRating,7117,0.680206
brand,5232,0.500048


In [22]:
print_missings(orders)

No missing values found.


### UNFOLD PROMOTIONS

In [27]:
items['promotion'].unique()

array([nan, '2018-07-13', '2018-07-01,2018-07-02,2018-07-13',
       '2018-07-05,2018-07-11', '2018-07-05', '2018-07-01', '2018-07-04',
       '2018-07-08', '2018-07-08,2018-07-13', '2018-07-09', '2018-06-30',
       '2018-07-11', '2018-07-06', '2018-07-08,2018-07-11',
       '2018-06-30,2018-07-13', '2018-06-30,2018-07-01,2018-07-11',
       '2018-07-06,2018-07-13', '2018-07-06,2018-07-10', '2018-07-10',
       '2018-07-08,2018-07-10', '2018-06-30,2018-07-04',
       '2018-07-01,2018-07-10', '2018-07-02', '2018-06-30,2018-07-01',
       '2018-07-04,2018-07-05,2018-07-06', '2018-07-04,2018-07-05',
       '2018-07-01,2018-07-06', '2018-06-30,2018-07-06',
       '2018-07-05,2018-07-13', '2018-07-01,2018-07-02', '2018-07-03',
       '2018-07-03,2018-07-04,2018-07-05',
       '2018-07-03,2018-07-04,2018-07-06', '2018-07-03,2018-07-04',
       '2018-07-03,2018-07-05', '2018-07-04,2018-07-09',
       '2018-07-04,2018-07-06', '2018-07-02,2018-07-04', '2018-07-12',
       '2018-07-02,2018-07-0

In [28]:
# split promotion feature
items = split_nested_features(items, split_vars = 'promotion', sep = ',')
items.head()

Added 3 split-based features.


,itemID,simulationPrice,brand,manufacturer,customerRating,category1,category2,category3,recommendedRetailPrice,promotion_0,promotion_1,promotion_2
0,1,3.43,NaN,1,4.38,1,1,1,8.84,NaN,NaN,NaN
1,2,9.15,NaN,2,3.00,1,2,1,16.92,NaN,NaN,NaN
2,3,14.04,NaN,3,5.00,1,3,1,15.89,NaN,NaN,NaN
3,4,14.10,NaN,2,4.44,1,2,1,40.17,NaN,NaN,NaN
4,5,7.48,NaN,2,2.33,1,1,1,17.04,NaN,NaN,NaN


In [29]:
# convert date types
promotion_vars = items.filter(like = 'promotion_').columns
for var in promotion_vars:
    items[var] = pd.to_datetime(items[var], infer_datetime_format = True)
    
items.dtypes

itemID                            object
simulationPrice                  float64
brand                             object
manufacturer                      object
customerRating                   float64
category1                         object
category2                         object
category3                         object
recommendedRetailPrice           float64
promotion_0               datetime64[ns]
promotion_1               datetime64[ns]
promotion_2               datetime64[ns]
dtype: object

# EXPORT

In [30]:
# save data frame
# save_csv_version() automatically adds version number to prevent overwriting
os.makedirs('/content/drive/MyDrive/DMC_2020/source_code/data/prepared', exist_ok=True)
save_csv_version('/content/drive/MyDrive/DMC_2020/source_code/data/prepared/orders.csv', orders, index = False, compression = 'gzip')
save_csv_version('/content/drive/MyDrive/DMC_2020/source_code/data/prepared/items.csv',  items,  index = False, compression = 'gzip')
print(orders.shape)
print(items.shape)

Saved as /content/prepared/orders_v1.csv
Saved as /content/prepared/items_v1.csv
(2181955, 5)
(10463, 12)
